In [1]:
import logging, sys, math
import logging, sys, math,os

currentFolder = os.path.abspath('')
try:
    sys.path.remove(str(currentFolder))
except ValueError: # Already removed
    pass

projectFolder = '/home/exx/muktadir/earthquakePrediction/'
projectFolder = 'F:/myProjects/tfKeras/UCSC/CMPS242/earthquake/'
sys.path.append(str(projectFolder))

In [2]:
exec(open("initKeras.py").read())

Using TensorFlow backend.


In [40]:
if sys.modules.get( 'library.SingleBinDataGenerator', False ) != False :
    del sys.modules['library.SingleBinDataGenerator'] 
if sys.modules.get( 'SingleBinDataGenerator', False ) != False :
    del sys.modules['SingleBinDataGenerator'] 
from library.SingleBinDataGenerator import *

logging.warning( "SingleBinDataGenerator loaded" )

trainGenerator = SingleBinDataGenerator(numBins = 10000, shuffle=False)
validationGenerator = SingleBinDataGenerator(startBinId = 10001, numBins = 1000, shuffle=False)


In [41]:

batch_size = 32
train_steps = int(10000 / batch_size)
validation_steps = int(1000 / batch_size)

In [17]:
model_input = layers.Input( shape = ( 64, 64, 1 ) )
x = layers.Conv2D(64, 
        kernel_size = (5,5), 
        activation = activations.relu,
        padding = 'same'
        )(model_input)
x = layers.Dropout(0.2)(x)
x = layers.Conv2D(16, kernel_size=(3,3), activation=activations.relu, padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(1)(x)

model = models.Model(model_input, x, name = "TinyCNN")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 64, 64)        1664      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 16)        9232      
_________________________________________________________________
flatten_2 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65537     
Total params: 76,433
Trainable params: 76,433
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(optimizer=optimizers.Adam(lr=0.001),
             loss = losses.MSE,
             metrics = [metrics.MSE, metrics.MAE])

In [ ]:
history = model.fit_generator( generator=trainGenerator,
                    validation_data = trainGenerator,
                    use_multiprocessing=True, 
                    workers=2, 
                    initial_epoch = 10,
                    epochs=100,
                    max_q_size = 100,
                    steps_per_epoch = train_steps,
                    validation_steps = validation_steps
                   )

D:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<library.S..., validation_data=<library.S..., use_multiprocessing=True, workers=2, initial_epoch=10, epochs=100, steps_per_epoch=312, validation_steps=31, max_queue_size=100)`
  if __name__ == '__main__':


Epoch 11/100
312/312 [==============================] - 10s 33ms/step - loss: 10.1909 - mean_squared_error: 10.1909 - mean_absolute_error: 2.6703 - val_loss: 27.3416 - val_mean_squared_error: 27.3416 - val_mean_absolute_error: 5.1328
Epoch 12/100
199/312 [==================>...........] - ETA: 4s - loss: 9.9220 - mean_squared_error: 9.9220 - mean_absolute_error: 2.6097  

In [ ]:

%matplotlib auto
import seaborn as sns
sns.set(style="darkgrid")

In [ ]:
trainError = history.history['mean_squared_error']
validationError = history.history['val_mean_squared_error']
n = list(range(len(trainError)))
ax = sns.lineplot(n, y=trainError, estimator = None, label = "train" )
ax = sns.lineplot(n, y=validationError, estimator = None, label = "validation" )

In [ ]:
trainError